## 🧪 Prueba de Little (MCAR) desde cero

---

### 👨‍🏫 ¿Qué busca la prueba?

La prueba de **Little (1988)** evalúa si los valores faltantes son **completamente aleatorios** (**MCAR**), usando un enfoque similar a una **chi-cuadrada multivariada** sobre las medias y covarianzas de los datos.

### 📋 Idea central:

Se basa en:

* Separar los datos en **patrones de ausencia de datos**
* Calcular medias y covarianzas por patrón
* Comparar esas estadísticas con el total usando una fórmula de **estadístico chi-cuadrado**

---

## 🔢 Paso a paso (con explicación):

### 1. **Identificar los patrones de valores faltantes**

```python
import pandas as pd
import numpy as np

# Dataset de ejemplo
df = pd.DataFrame({
    'x1': [1.2, 2.4, np.nan, 1.5, 2.1],
    'x2': [3.1, 2.9, 3.5, np.nan, 3.0],
    'x3': [np.nan, 4.0, 4.2, 4.1, 4.3]
})

# Crear un patrón por fila basado en valores faltantes
patterns = df.isnull().astype(int)
df['pattern'] = patterns.apply(lambda row: ''.join(row.astype(str)), axis=1)
```

Esto te da, por ejemplo:

```
pattern:
101
000
010
100
000
```

---

### 2. **Calcular la media y covarianza total (para datos completos)**

```python
# Solo usamos las filas completas para calcular la "media poblacional"
complete_cases = df[df['pattern'] == '000'].drop(columns='pattern')

mu = complete_cases.mean()
Sigma = complete_cases.cov()
Sigma_inv = np.linalg.inv(Sigma)
```

Esto es importante porque **comparamos cada patrón de datos faltantes contra estos parámetros "generales"**.

---

### 3. **Para cada patrón, computar su estadístico chi-cuadrado**

Fórmula:

$$
\chi^2 = \sum_{g=1}^{G} n_g \cdot (\bar{x}_g - \mu_g)^T \Sigma_g^{-1} (\bar{x}_g - \mu_g)
$$

Donde:

* $g$ es cada patrón
* $n_g$: número de observaciones con ese patrón
* $\bar{x}_g$: vector media de ese patrón
* $\mu_g$: vector media total solo en las variables disponibles
* $\Sigma_g^{-1}$: submatriz de la inversa de covarianza correspondiente a las variables presentes

---

### 4. **Implementación del cálculo completo**

```python
chi2 = 0
df_total = df.drop(columns='pattern')

for pattern, group in df.groupby('pattern'):
    observed_vars = [col for col, flag in zip(df_total.columns, list(pattern)) if flag == '0']
    
    if len(observed_vars) == 0 or len(group) < 2:
        continue  # no puedes calcular media o covarianza con menos de 2 filas

    x_g_bar = group[observed_vars].mean()
    mu_g = mu[observed_vars]
    Sigma_g = Sigma.loc[observed_vars, observed_vars]
    Sigma_g_inv = np.linalg.inv(Sigma_g)
    
    diff = (x_g_bar - mu_g).values.reshape(-1, 1)
    chi2 += len(group) * float(diff.T @ Sigma_g_inv @ diff)
```

---

### 5. **Grados de libertad y p-valor**

$$
\text{gl} = \sum_{g} p_g - p
$$

Donde:

* $p_g$ = número de variables observadas en patrón $g$
* $p$ = número total de variables

```python
from scipy.stats import chi2 as chi2_dist

df_list = []
for pattern, group in df.groupby('pattern'):
    p_g = sum([1 for i in pattern if i == '0'])
    df_list.append(p_g)

total_df = sum(df_list) - len(df_list) * df_total.shape[1]
p_value = 1 - chi2_dist.cdf(chi2, df=total_df)
```

---

## ✅ Resultado final

```python
print(f"Estadístico Chi-cuadrado: {chi2:.4f}")
print(f"Grados de libertad: {total_df}")
print(f"Valor p: {p_value:.4f}")

if p_value > 0.05:
    print("✅ No se rechaza MCAR. Los valores faltantes podrían ser completamente aleatorios.")
else:
    print("❌ Se rechaza MCAR. Los valores faltantes no parecen completamente aleatorios (probablemente MAR o MNAR).")
```

---

## 📌 Conclusión

Esto reproduce la lógica de la **Prueba de Little** *sin usar librerías externas* (excepto `scipy.stats.chi2` para calcular el p-valor, aunque también podrías hacerlo con tablas).

¿Te gustaría que convierta todo esto a un solo bloque limpio para tu `.ipynb`, con comentarios inline y ejemplos listos para correr?
